In [1]:
!pip install llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface transformers accelerate bitsandbytes llama-index-readers-web chainlit langchain-community python-dotenv unstructured torch nltk

INFO: pip is looking at multiple versions of llama-index-llms-huggingface to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of llama-index-embeddings-huggingface to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-index-readers-web to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-index

In [2]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/52yxo196e134l58trhjfm/Brief-history-of-Vietnam.zip?rlkey=u0bsxff2w6y8y6bpn8whfea07&st=1aqde08r&dl=1" -O data/Brief-history-of-Vietnam.zip
!unzip data/Brief-history-of-Vietnam.zip -d data

--2024-11-18 12:56:31--  https://www.dropbox.com/scl/fi/52yxo196e134l58trhjfm/Brief-history-of-Vietnam.zip?rlkey=u0bsxff2w6y8y6bpn8whfea07&st=1aqde08r&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc51f34818bd151d65bab6fb5c01.dl.dropboxusercontent.com/cd/0/inline/Cek7O5fceMecVmSVG68PLUK80PX_oOFSaGe_AgtOcfHh5vIf_sKUpMGnaYPDQGG_J6IyX1eEtlA8jaNyfwu2PK3DS-denbUX_kOtsyFCS8DekTcB8SbWr3PwpMb0dY35d4B4xrhMNTAyWgOi-siJ3Yvz/file?dl=1# [following]
--2024-11-18 12:56:32--  https://uc51f34818bd151d65bab6fb5c01.dl.dropboxusercontent.com/cd/0/inline/Cek7O5fceMecVmSVG68PLUK80PX_oOFSaGe_AgtOcfHh5vIf_sKUpMGnaYPDQGG_J6IyX1eEtlA8jaNyfwu2PK3DS-denbUX_kOtsyFCS8DekTcB8SbWr3PwpMb0dY35d4B4xrhMNTAyWgOi-siJ3Yvz/file?dl=1
Resolving uc51f34818bd151d65bab6fb5c01.dl.dropboxusercontent.com (uc51f34818bd151d65bab6fb5c01.dl.dr

In [ ]:
!pip install --upgrade torch

In [3]:
# %%
import torch
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.readers.file import UnstructuredReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine

# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.float16,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
# )


def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  # ensure we start with a system prompt, insert blank if needed
  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt

  # add final assistant prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt


llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-beta",
    tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3900,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
   # model_config={"protected_namespaces": ()}
)

#llm.model.config.protected_namespaces = ()

# %%
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.llm = llm
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


# %%
from llama_index.readers.file import UnstructuredReader
from pathlib import Path
import nltk

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')

years = [1975, 1969, 1945]

loader = UnstructuredReader()
doc_set = {}
all_docs = []
for year in years:
    year_docs = loader.load_data(
        file=Path(f"./data/War/{year}.htm"), split_documents=False
    )
    # insert year metadata into each year
    for d in year_docs:
        d.metadata = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

print(f"Loaded {len(all_docs)} documents")


# %%
# initialize simple vector indices
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings

Settings.chunk_size = 512
index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults()
    cur_index = VectorStoreIndex.from_documents(
        doc_set[year],
        storage_context=storage_context,
    )
    index_set[year] = cur_index
    storage_context.persist(persist_dir=f"./storage/{year}")


# %%
# Load indices from disk
from llama_index.core import load_index_from_storage

index_set = {}
for year in years:
    storage_context = StorageContext.from_defaults(
        persist_dir=f"./storage/{year}"
    )
    cur_index = load_index_from_storage(
        storage_context,
    )
    index_set[year] = cur_index


# %%
from llama_index.core.tools import QueryEngineTool, ToolMetadata

individual_query_engine_tools = [
    QueryEngineTool(
        query_engine=index_set[year].as_query_engine(),
        metadata=ToolMetadata(
            name=f"vector_index_{year}",
            description=f"useful for when you want to answer queries about the {year} War of Vietnam",
        ),
    )
    for year in years
]


# %%
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=individual_query_engine_tools,
   llm=HuggingFaceLLM(model_name="HuggingFaceH4/zephyr-7b-alpha"),

)

# %%
query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine",
        description="useful for when you want to answer queries that require analyzing multiple War documents of Vietnam",
    ),
)

# %%
tools = individual_query_engine_tools + [query_engine_tool]

# %%
from llama_index.core.response.notebook_utils import display_response
for year in index_set:
  query_engine = index_set[year].as_query_engine(response_mode="refine")


response = query_engine.query("What happened in 3 January in Vietnam war in 1975?")
display_response(response)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


**`Final Response:`** On January 3, 1946, the Viet Minh launched a large-scale attack on the 14/13 Frontier Force Rifles camp at Biên Hòa in southern Vietnam. Around 900 Viet Minh fighters stormed the British and Indian-held camp, but the well-prepared defenders successfully repelled the attack without any losses. The Viet Minh suffered around 100 casualties, mostly from British machine-gun crossfire. This battle marked the last major confrontation between the British/Indian, French, and Japanese forces and the Viet Minh during the 1945-1946 War in Vietnam. After this defeat, the Viet Minh shifted their tactics to unconventional warfare, such as ambushes, hit-and-run raids, and assassinations. The British, French, and Japanese forces continued to patrol and conduct security sweeps in the region until the French took over the theater of operations in late January 1946. The British forces withdrew into Saigon, and the British commander, Gracey, left Vietnam on January 28, 1946.

In [ ]:
!pip install chainlit